In [1]:
%matplotlib inline
%matplotlib widget
from datetime import datetime, timedelta 
import time
from astropy.time import Time
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from copy import copy

import asyncio
import os
from lsst.ts import salobj 
from scipy.fft import fft, fftfreq
from lsst.ts.m2com import NUM_ACTUATOR, NUM_TANGENT_LINK

from lsst_efd_client import EfdClient


In [2]:
domain = salobj.Domain()
m2 = salobj.Remote(domain, "MTM2")
await m2.start_task
await m2.cmd_setLogLevel.set_start(level=10)

In [ ]:
"""    
DISABLED = 1
ENABLED = 2
FAULT = 3
OFFLINE = 4
STANDBY = 5

Standby  -->  Disable  -->  Enabled

await m2.cmd_start.set_start(timeout=30)
await m2.cmd_enable.set_start(timeout=200)

3) To Stop, do this:    Enabled  -->  Disable  -->  Standby

await m2.cmd_disable.set_start(timeout=30)
await m2.cmd_standby.set_start(timeout=30) """ 

In [49]:
# get status
state = m2.evt_summaryState.get()
print(state)

private_revCode: 0ebebdcc, private_sndStamp: 1684936362.83504, private_rcvStamp: 1684936362.8355777, private_seqNum: 23, private_identity: MTM2, private_origin: 11350, summaryState: 2


In [46]:
# Standby  -->  Disable
await m2.cmd_start.set_start(timeout=30)

In [48]:
# Disable  -->  Enabled
await m2.cmd_enable.set_start(timeout=200)

In [48]:
# Enabled  -->  Disable
await m2.cmd_disable.set_start(timeout=30)

In [42]:
#Disable  -->  Standby
await m2.cmd_standby.set_start(timeout=30)

In [ ]:
#Fault --> Standby
await m2.cmd_standby.set_start(timeout=30)

In [3]:
def retrieve_efd_client(is_summit=True):
    """
    Retrieves a client to EFD.

    Parameters
    ----------
    is_summit : bool, optional
        This notebook is running on the summit or not. If not, the returned object will point
        to the test stand at Tucson.

    Returns
    -------
    EfdClient : The interface object between the Nublado and summit/Tucson EFD.
    """
    efd_name = "summit_efd" if is_summit else "tucson_teststand_efd"        
    return EfdClient(efd_name)


In [6]:
efd_client = retrieve_efd_client(is_summit=False)

n_act = NUM_ACTUATOR

#fill a list with n actuators
n = NUM_TANGENT_LINK

actuators = [73, 74, 75, 76, 77, 78]


for idx in range(n):
    print(f"actuator: {actuators[idx]}")

actuator: 73
actuator: 74
actuator: 75
actuator: 76
actuator: 77
actuator: 78


In [18]:
# retrieve from EFD the last 20 seconds of measured forces on actuators

# increase the time to at least 120 sec, bettter if 180 sec    
await asyncio.sleep(15)    


time_end = datetime.now()
time_end = Time(time_end, scale="utc")
time_start = time_end - timedelta(seconds = 10)
time_start = Time(time_start, scale="utc")

names = list()
for idx in range(n_act):
    names.append(f"measured{idx}")
    
data_act_before = await efd_client.select_time_series(
        "lsst.sal.MTM2.axialForce",
        fields=names,
        start=time_start,
        end=time_end)

f_max = data_act_before.max()
f_min = data_act_before.min()
f_max_values = f_max.to_numpy()
f_min_values = f_min.to_numpy()
#print(f"f_max f_min : {f_max_values} {f_min_values}")

f_max f_min : [224.52188726829485 214.76653219548163 222.23556418102064
 220.21102554427432 215.8937965230037 352.6790181832579 218.0412897810235
 209.37500891286768 213.20571608812685 207.49289275161496
 223.6594497071341 215.68884333295583 214.9245225537258 216.68402517324571
 211.47312716053824 340.59937546832833 213.77827815067843
 214.35114899471523 212.413678086388 210.65711212553325 216.93727413421522
 210.21288781262916 220.07893139546547 215.01245496135726
 217.2440810805641 290.85388588613586 212.77394926378116
 219.79427481128468 218.87752288424755 227.09483763808635
 264.65026560087324 265.72921062484545 265.18310278800607
 258.19977439215967 253.95384539448744 263.08994856533735
 265.0640041829226 259.1989833441232 254.03909267920778 257.4225924523515
 261.0503868336491 252.767849288882 251.86074850380604 257.65821972747403
 260.3683939788948 258.6219913398265 256.81688295145875 268.23400462800583
 265.27044030983444 261.5941394983938 265.1120809249027 262.6602512206985
 2

In [20]:
f_max - f_min

measured0     0.767306
measured1     0.922703
measured2     0.902037
measured3     0.681378
measured4     0.707568
                ...   
measured73         NaN
measured74         NaN
measured75         NaN
measured76         NaN
measured77         NaN
Length: 78, dtype: object

In [80]:
#Check if during cart rotation the tangetial actuators overcome the limit

MAX_FORCE_ALLOWED = 8000
MIN_ROT_ANGLE = 0.2

# get time for starting and ending telemetry data
time_start1 = datetime.now()
time_start1 = Time(time_start1, scale="utc")
print(f"UTC time to is {time_start} now.")

zenithAngle = m2.tel_zenithAngle.get()
NewzenithAngle = copy(zenithAngle)
NewzenithAngle.inclinometerRaw = NewzenithAngle.inclinometerRaw + 10.0

data_measured_forces = m2.tel_axialForce.get()
data_measured_forces_values = np.array(data_measured_forces.measured, float)

#start cart rotation
command = input("start cart rotation")


while abs(zenithAngle.inclinometerRaw - NewzenithAngle.inclinometerRaw) > MIN_ROT_ANGLE :
    print(f"Raw inclinometer angle: {zenithAngle.inclinometerRaw:0.2f} degree.")
    print(f" Difference inclinometer angle: {abs(zenithAngle.inclinometerRaw - NewzenithAngle.inclinometerRaw)}")
    NewzenithAngle = m2.tel_zenithAngle.get()
    data_measured_forces = m2.tel_tangentForce.get()
    data_measured_forces_values = np.array(data_measured_forces.measured, float)  
    print(f"data_measured_forces_values : {data_measured_forces_values}")  
    if any(abs(data_measured_forces_values) > MAX_FORCE_ALLOWED):
        print(f"alarm : {data_measured_forces_values}")
        NewzenithAngle = copy(zenithAngle)
    else :
        NewzenithAngle = m2.tel_zenithAngle.get()
        print(f"Raw inclinometer angle: {zenithAngle.inclinometerRaw:0.2f} degree.")
        #print(f"tangent value : {data_measured_forces_values}")       
else:
    print(f"Test terminated")   
    

UTC time to is 2023-05-30 17:59:13.121580 now.


start cart rotation 


Raw inclinometer angle: 90.00 degree.
 Difference inclinometer angle: 10.0
data_measured_forces_values : [ 113.05523059 -157.44721394   30.48933482 -174.27811224   50.58798005
  -35.77996362]
Raw inclinometer angle: 90.00 degree.
Test terminated


In [73]:
from copy import copy

zenithAngle = m2.tel_zenithAngle.get()
print(f"Measured zenith angle: {zenithAngle.measured:0.2f} degree.")
print(f"Raw inclinometer angle: {zenithAngle.inclinometerRaw:0.2f} degree.")
print(f"Processed inclinometer angle: {zenithAngle.inclinometerProcessed:0.2f} degree.")

NewzenithAngle = copy(zenithAngle)
NewzenithAngle.inclinometerRaw = NewzenithAngle.inclinometerRaw +10.0

print(f"Measured zenith angle: {NewzenithAngle.measured:0.2f} degree.")
print(f"Raw inclinometer angle: {NewzenithAngle.inclinometerRaw:0.2f} degree.")
print(f"Processed inclinometer angle: {NewzenithAngle.inclinometerProcessed:0.2f} degree.")

Measured zenith angle: 0.04 degree.
Raw inclinometer angle: 89.96 degree.
Processed inclinometer angle: 89.10 degree.
Measured zenith angle: 0.04 degree.
Raw inclinometer angle: 99.96 degree.
Processed inclinometer angle: 89.10 degree.


In [33]:
# get time for starting and ending telemetry data
time_start1 = datetime.now()
time_start1 = Time(time_start1, scale="utc")
print(f"UTC time to is {time_start} now.")

await asyncio.sleep(20)    

time_end1 = datetime.now()
time_end1 = Time(time_end1, scale="utc")


print(f"time start : {time_start1}")
print(f"time end : {time_end1}")

UTC time to is 2023-05-24 13:40:27.881750 now.
time start : 2023-05-24 13:44:15.333102
time end : 2023-05-24 13:44:35.334001


In [34]:
# switch off actuators power

#wait for pushing red button
command = input("Cut the power")

Cut the power 


In [35]:
# retriveve telemetry with cut power

# get time for starting and ending telemetry data
time_start2 = datetime.now()
time_start2 = Time(time_start2, scale="utc")
print(f"UTC time to is {time_start2} now.")

await asyncio.sleep(120)    

time_end2 = datetime.now()
time_end2 = Time(time_end2, scale="utc")


#retrieve telemetry
names = list()
for idx in range(72):
    names.append(f"measured{idx}")    
    
data_act = await efd_client.select_time_series(
        "lsst.sal.MTM2.axialForce",
        fields=names,
        start=time_start1,
        end=time_end2)
data_act.to_csv("actuators_data.csv")

data_power_cell = await efd_client.select_time_series(
        "lsst.sal.MTM2.powerStatus",
        fields=["motorVoltage","motorCurrent","commVoltage","commCurrent"],
        start=time_start1,
        end=time_end2)
data_power_cell.to_csv("power_data.csv")


print(f"time start : {time_start1}")
print(f"time end : {time_end2}")

UTC time to is 2023-05-24 13:44:46.262101 now.
time start : 2023-05-24 13:44:15.333102
time end : 2023-05-24 13:46:46.262786


In [38]:
#wait for restoring the power
command = input("Restore the power")

#await m2.cmd_standby.set_start(timeout=30)

Restore the power 


In [39]:
#check the status
state = m2.evt_summaryState.get()
print(state)

private_revCode: 0ebebdcc, private_sndStamp: 1684934561.384845, private_rcvStamp: 1684935457.9080932, private_seqNum: 19, private_identity: MTM2, private_origin: 11350, summaryState: 2


In [101]:
await m2.cmd_start.set_start(timeout = 30)

state = m2.evt_summaryState.get()
print(state)

private_revCode: 0ebebdcc, private_sndStamp: 1684521343.9021816, private_rcvStamp: 1684521343.902533, private_seqNum: 16, private_identity: MTM2, private_origin: 36543, summaryState: 1


In [102]:
await m2.cmd_enable.set_start(timeout = 200)

state = m2.evt_summaryState.get()
print(state)

private_revCode: 0ebebdcc, private_sndStamp: 1684521380.6627302, private_rcvStamp: 1684521380.663149, private_seqNum: 17, private_identity: MTM2, private_origin: 36543, summaryState: 2


In [50]:
#in principle this command is exectuted at the restirong of the system and it is not necessary
#bring all actuators to zero   
n_act = NUM_ACTUATOR - NUM_TANGENT_LINK
all_actuators = [0] * n_act
f_Delta = [0] * n_act
f_Delta = np.array(f_Delta, float)
forces = f_Delta

#apply force of the loop
for idx in range(n_act):
    #print(f"command fDelta to actuator: {idx}.")
    force = forces[idx]
# change the below function with MTM2_command_resetForceOffsets    
await m2.cmd_resetForceOffsets.set_start()
await asyncio.sleep(5)
print("Actuators at zero")

Actuators at zero
